In [1]:
import h5py
import numpy as np
import pandas as pd
# f = h5py.File("travel_time.hdf5", "w")

In [2]:
from util import *
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [ ]:
dataset = load_dataset()

In [ ]:
f = h5py.File("travel_time_second.hdf5", "w")
for n, g in dataset.groupby('link'):
    g.index = g.dt
    g = g.sort_index()
    history_mean = g.groupby([lambda x: (x.isoweekday(), x.hour, x.minute)])['travel_time'].mean()

    hour_minute_mean = g.groupby([lambda x: (x.hour, x.minute)])['travel_time'].mean()
    
    hour_mean = g.groupby([lambda x: (x.hour)])['travel_time'].mean()

    travel_time_feature = g['travel_time'].describe()

    train_test_split_date = ['20170301', '20170525', '20170531']
    train, cv = g[train_test_split_date[0]:train_test_split_date[1]], \
                g[train_test_split_date[1]:train_test_split_date[2]]
    
    test = g['20170601':'20170630']
    train_ds, train_ts = load_data(train, travel_time_feature, history_mean, hour_minute_mean, hour_mean)
    cv_ds, cv_ts = load_data(cv, travel_time_feature, history_mean, hour_minute_mean, hour_mean,
                            datetime(2017, 5, 25), datetime(2017, 5, 31))
    f.create_group(n)
    f[n].create_group('train')
    f[n].create_group('test')
    f[n].create_group('cv')
    f[n]['cv'].create_dataset('data', data=np.array(cv_ds))
    f[n]['cv'].create_dataset('time', data=np.array(cv_ts))
    f[n]['train'].create_dataset('data', data=np.array(train_ds))
    f[n]['train'].create_dataset('time', data=np.array(train_ts))
    
    test_ds, test_ts = load_data(test, travel_time_feature, history_mean, 
                             hour_minute_mean, hour_mean, datetime(2017, 6, 1, 6), datetime(2017, 6, 30, 9))
    f[n]['test'].create_dataset('data', data=np.array(test_ds))
    f[n]['test'].create_dataset('time', data=np.array(test_ts))
    
f.close()

In [20]:
train.head()

,link,date,interval,travel_time,dt
dt,,,,,
2017-03-01 00:00:00,3377906282328510514,2017-03-01,"[2017-03-01 00:00:00,2017-03-01 00:02:00)",3.0,2017-03-01 00:00:00
2017-03-01 00:02:00,3377906282328510514,2017-03-01,"[2017-03-01 00:02:00,2017-03-01 00:04:00)",3.0,2017-03-01 00:02:00
2017-03-01 00:04:00,3377906282328510514,2017-03-01,"[2017-03-01 00:04:00,2017-03-01 00:06:00)",3.1,2017-03-01 00:04:00
2017-03-01 00:08:00,3377906282328510514,2017-03-01,"[2017-03-01 00:08:00,2017-03-01 00:10:00)",2.4,2017-03-01 00:08:00
2017-03-01 00:10:00,3377906282328510514,2017-03-01,"[2017-03-01 00:10:00,2017-03-01 00:12:00)",2.4,2017-03-01 00:10:00


In [12]:
np.unique(dataset.link).shape

(132,)

In [133]:
g = dataset[dataset.link=='9377906285615510514']
g.index = g.dt
g.groupby([lambda x: (x.hour)])['travel_time'].mean()[9]

5.7459930313588847

In [8]:
f = h5py.File('travel_time_second.hdf5', 'r')
print f.keys()
f.close()

[u'3377906280028510514', u'3377906280395510514', u'3377906281518510514', u'3377906281774510514']


In [ ]:
f['9377906285566510514']['cv']['time'].value.shape

In [ ]:
train_ds, train_ts = load_data(train, travel_time_feature, history_mean, hour_minute_mean)

In [24]:
f.close()